In [21]:
import pandas as pd
import numpy as np
from numpy import sin,cos,tan,cosh,sinh,tanh,deg2rad,sqrt

input_data = pd.read_csv("uls_lb.csv", skiprows=6, skipfooter=8,
usecols=[0,2,4,5,6,7,8,9,10],
# na_values="-"
)
input_data.rename(columns={"Unnamed: 4": "load_combination","Unnamed: 2": "column","Unnamed: 0": "support", "Fvert\n[kN]":"Fv", "Fmajor\n[kN]":"Fy","Fminor\n[kN]":"Fx","Mmajor\n[kNm]":"My","Mminor\n[kNm]":"Mx","Mtor\n[kNm]":"Mtor"}, inplace=True)
input_data.ffill(inplace=True)

def column_sizes(input):
    """
    Function to re-write the column string in a more readable way
    """
    return input.split('(')[1].split(')')[0]
    
# Apply the column_sizes function to the imported data. 
input_data.column = input_data.column.apply(column_sizes)

# View the input_data
input_data.tail(20)


C:\Users\rmacdonald.BLYTHANDBLYTH\AppData\Local\Temp\ipykernel_5520\1845024226.py:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  input_data = pd.read_csv("sls_lb.csv", skiprows=6, skipfooter=8,


,support,column,load_combination,Fv,Fy,Fx,My,Mx,Mtor
820,z,UC 203x203x86,101 (Final) Cb4.13-1.2D+1.2I+1.2S+1.2W,0.0,0.7,2.4,0.0,0.0,0.0
821,z,UC 203x203x86,102 (Final) Cb4.14-1.2D+1.2I+1.2S+1.2W,0.0,0.7,2.4,0.0,0.0,0.0
822,z,UC 203x203x86,103 (Final) Cb4.15-1.2D+1.2I+1.2S+1.2W,0.0,0.4,4.0,0.0,0.0,0.0
823,z,UC 203x203x86,104 (Final) Cb4.16-1.2D+1.2I+1.2S+1.2W,0.0,0.4,4.0,0.0,0.0,0.0
824,z,UC 203x203x86,105 (Final) Cb5.1-D+1.4W,0.0,-1.0,-5.3,0.0,0.0,0.0
825,z,UC 203x203x86,106 (Final) Cb5.2-D+1.4W,0.0,-1.0,-5.3,0.0,0.0,0.0
826,z,UC 203x203x86,107 (Final) Cb5.3-D+1.4W,0.0,-1.5,-2.8,0.0,0.0,0.0
827,z,UC 203x203x86,108 (Final) Cb5.4-D+1.4W,0.0,-1.5,-2.8,0.0,0.0,0.0
828,z,UC 203x203x86,109 (Final) Cb5.5-D+1.4W,0.0,-0.3,4.0,0.0,0.0,0.0
829,z,UC 203x203x86,110 (Final) Cb5.6-D+1.4W,0.0,-0.3,4.0,0.0,0.0,0.0


In [22]:
# define support class
class Support():
    def __init__(self, df, length, width):
        self.df = df
        self.calculate_initial_length_and_width()
        self.set_max_loading()        
        self.optimise_area(increment_size=0.01)

        self.starting_length = self.df
        self.bearing_pressure = None
        self.bearing_capacity = None 

    
    def calculate_initial_length_and_width(self,):
        """
        Uses the data in the df attribute to calculate initial length, width and area values and stores them within the object. 
        """
        assert len(self.df.column.unique()) == 1, print(f"Not all of the column values are the same")

        length = int(self.df.column.iloc[0][3:].split('x')[0])/1000
        width = int(self.df.column.iloc[0][3:].split('x')[1])/1000 #changing mm to m (/1000)
        self.length = length
        self.width = width
        self.area = length * width        

    def optimise_area(self, increment_size):
        self.bearing_capacity=150 # make input nicer... ask user for input?        
        self.bearing_pressure=self.calculate_bearing_pressure()
        print(f"Starting bearing pressure {self.bearing_pressure}, TRYING AREA {self.area}")
        iteration_number = 1 # track how many iterations we needed to get the optimal area...
        while self.bearing_pressure > self.bearing_capacity:
            iteration_number = iteration_number + 1
            self.length = self.length+increment_size
            self.width = self.width+increment_size
            self.area = self.width * self.length
            self.bearing_pressure = self.calculate_bearing_pressure()
        print(f"Found optimal area in {iteration_number} iterations")
        print(f"Calculated bearing pressure {self.bearing_pressure}, TRYING AREA {self.area:.2f} (L={self.length:.2f}m, W={self.width:.2f}m)")

        # can just do force/pressure to get maximum area then work out l, b beginning from there? most important bit is to figure out sliding
        # capacity for each of these, therefore will maybe be based on that
        # where should depth come into it - minimum depth that is okay for bearing pressure and sliding? Bearing pressure will never change, therefore can define that based on maximum FV, then use sliding to define L and B but always has to be less than calculated area, however depth can change
        print(f"Minimum length is {self.length:.2f}, minimum width is {self.width:.2f}\n") # 2f - format to 2 d.p.

    def loop_all_loading(self):
        self.set_loading()
        # bearing pressure calc to be renamed as 'set_important_parameters' or similar. 
        self.calculate_bearing_pressure()
    def set_max_loading(self):
        """
        Based on the DF, set every load combination, then calculate the bearing pressure. Remember what the max loading condition is and set that. 
        """
        max_bearing_pressure = 0 # initialise as 0 
        for i in range(0,len(self.df)):
            self.P=self.df.iloc[i].Fv
            self.H_Y=self.df.iloc[i].Fy #df.FY
            self.H_X=self.df.iloc[i].Fx
            self.M_X=self.df.iloc[i].Mx
            self.M_Y=self.df.iloc[i].My
            # only update max_bearing pressure if it beats the last loading case... 
            bearing_pressure = self.calculate_bearing_pressure()
            if bearing_pressure > max_bearing_pressure:
                max_bearing_pressure = bearing_pressure
                max_loading_case = i 
                   
        print(f"max loading case {max_loading_case}")
        print(f"max bearing pressure {max_bearing_pressure}")
            
        self.P=self.df.iloc[max_loading_case].Fv
        self.H_Y=self.df.iloc[max_loading_case].Fy #df.FY
        self.H_X=self.df.iloc[max_loading_case].Fx
        self.M_X=self.df.iloc[max_loading_case].Mx
        self.M_Y=self.df.iloc[max_loading_case].My
        print(self.df.iloc[max_loading_case])
        # Now set the max loading on the object... 

        



    def calculate_bearing_pressure(self):    
        # density of concrete (kN/m3)
        P_CONCRETE=23.6
        # depth of pad footing (m)
        H=0.750
        # depth of soil over pad (m)
        H_SOIL=0.500 #m
        P_SOIL=0.20 #m
        SHEAR_STRENGTH=deg2rad(25) #rad
        BASE_FRICTION=deg2rad(19.3) #rad
        P_BEARING=150 #kn/m2

        # column details
        I_A=0.850 #m base length
        B_A=0.450 #m base width
        E_PXA=0 #m eccentrincity in x
        E_PYA=0 #m ecc in y


        # Dead surcharge load (kN/m2)
        F_Gsur=0 
        F_Qsur=0

        # Pad footing self weight
        F_SWT=H*P_CONCRETE
        # soil self weight
        F_SOIL=H_SOIL*P_SOIL
        # total foundation load
        F=self.area*(F_SOIL+F_Qsur+F_SOIL+F_Gsur)



        # Stability against sliding (kN)
        H_friction=max((self.P+(F_Gsur+F_Qsur+F_SWT),0)) #resistance due to base friction
        K_P=(1+sin(SHEAR_STRENGTH))/(1-sin(SHEAR_STRENGTH)) #passive pressure coef
        H_D=sqrt(self.H_Y**2+self.H_X**2)

        # FIX H_X... 

        if self.H_X==0:
            ALPHA=0
        else:
            ALPHA=self.area*tan(self.H_Y/self.H_X)
        H_xpas=0.5*K_P*(H**2+2*H*H_SOIL)*self.width*P_SOIL  # BWCKY CHECK EQUATION IS STILL RIGHT!!
        H_ypas=0.5*K_P*(H**2+2*H*H_SOIL)*self.length*P_SOIL
        # resultant passive resistance
        H_RESULTANT=abs(H_xpas*cos(ALPHA))+abs(H_ypas*sin(ALPHA))
        
        # self.h_res_test()
        H_RES=H_friction+H_RESULTANT #should be greater than resultant horizontal load then PASS


        # STABILITY AGAINST TURNING
        # self.m_x_res_test() # check if fail or pass... 
        M_XOT=self.M_X+self.H_X*H
        M_XSUR=self.area*(F_Gsur+F_SWT+F_SOIL)*self.length/2
        M_XAXIAL=self.P*(self.length/2-E_PXA)
        M_XRES=M_XSUR+M_XAXIAL

        M_YOT=self.M_Y+self.H_Y*H
        M_YSUR=self.area*(F_Gsur+F_SWT+F_SOIL)*self.width/2
        M_YAXIAL=self.P*(self.width/2-E_PYA)
        M_YRES=M_YSUR+M_YAXIAL

        # Pad base reactions
        T=F+self.P
        E_TX=(self.P*E_PXA+self.M_X+self.H_X*H)/T
        E_TY=(self.P*E_PYA+self.M_Y+self.H_Y*H)/T

        Q_1=T/self.area-6*T*E_TX/(self.length*self.area)-6*T*E_TY/(self.width*self.area)
        Q_2=T/self.area-6*T*E_TX/(self.length*self.area)+6*T*E_TY/(self.width*self.area)
        Q_3=T/self.area+6*T*E_TX/(self.length*self.area)-6*T*E_TY/(self.width*self.area)
        Q_4=T/self.area+6*T*E_TX/(self.length*self.area)+6*T*E_TY/(self.width*self.area)

        Q_MIN=min(Q_1,Q_2,Q_3,Q_4)
        Q_MAX=max(Q_1,Q_2,Q_3,Q_4) #Is maximum base pressure less than the allowable
        bearing_pressure = Q_MAX
        return bearing_pressure


In [26]:
support_dictionary= {}
# New dataframes created for each different support type. 
# These dataframes are then used to create independent Support() objects, with different dataframes.

support_size=input_data.support.unique()

for support_name in support_size:
    independent_support_data=input_data.loc[input_data['support']==support_name]
    support_dictionary[support_name]=Support(df=independent_support_data, length=1000, width = 1000)

print(f"Support dict keys {support_dictionary.keys()}")

max loading case 174
max bearing pressure 83038.41191673778
support                                    A
column                         UC 203x203x86
load_combination    109 (Final) Cb5.5-D+1.4W
Fv                                     136.7
Fy                                    -147.9
Fx                                       0.3
My                                       0.0
Mx                                       0.0
Mtor                                     0.0
Name: 174, dtype: object
Starting bearing pressure 83038.41191673778, TRYING AREA 0.041209
Found optimal area in 164 iterations
Calculated bearing pressure 149.17211998195225, TRYING AREA 3.36 (L=1.83m, W=1.83m)
Minimum length is 1.83, minimum width is 1.83

max loading case 350
max bearing pressure 44883.635119331026
support                                                 B
column                                      UC 203x203x86
load_combination    99 (Final) Cb4.11-1.2D+1.2I+1.2S+1.2W
Fv                                       

In [27]:
# Empty df for storing the summary table
summary_table_df=pd.DataFrame()
# Get the largest Fv Fy and Fx and smallest Fv for all of the supports. 
for key, support in support_dictionary.items():
        for i in ['Fv','Fy','Fx']:
            if i == 'Fv':
                support.df.sort_values(by=i, ascending=False, inplace=True)
                summary_table_df = summary_table_df.append(support.df.iloc[0])
                support.df.sort_values(by=i, ascending=True, inplace=True)
                summary_table_df = summary_table_df.append(support.df.iloc[0])
            else:
                support.df.sort_values(by=i, ascending=False, inplace=True, key=abs)
                summary_table_df = summary_table_df.append(support.df.iloc[0])

summary_table_df.head(10)
# assert len(summary_table_df) == 12, "Ahhhh noooo"
report_df=summary_table_df.copy()
report_df.set_index('support', inplace=True)
report_df.rename({'load_combination':'Load Combination'},axis='columns', inplace=True)
print(report_df.to_latex(columns=['Load Combination','Fv','Fy','Fx'],index_names=False))

C:\Users\rmacdonald.BLYTHANDBLYTH\AppData\Local\Temp\ipykernel_5520\99942857.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  support.df.sort_values(by=i, ascending=False, inplace=True)
C:\Users\rmacdonald.BLYTHANDBLYTH\AppData\Local\Temp\ipykernel_5520\99942857.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  summary_table_df = summary_table_df.append(support.df.iloc[0])
C:\Users\rmacdonald.BLYTHANDBLYTH\AppData\Local\Temp\ipykernel_5520\99942857.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  support.df.sort_values(by=i

\begin{tabular}{llrrr}
\toprule
{} &                       Load Combination &     Fv &     Fy &    Fx \\
\midrule
A &                       60 Snow Drift 90 &  907.4 &   -0.5 &  -0.1 \\
A &               112 (Final) Cb5.8-D+1.4W &  136.7 & -147.7 &   0.2 \\
A &               109 (Final) Cb5.5-D+1.4W &  136.7 & -147.9 &   0.3 \\
A &               113 (Final) Cb5.9-D+1.4W &  273.8 &    2.0 &  -7.1 \\
B &                       60 Snow Drift 90 &  543.0 &   -0.8 &  -7.3 \\
B &              119 (Final) Cb5.15-D+1.4W &   42.5 &    3.3 &  -6.8 \\
B &   95 (Final) Cb4.7-1.2D+1.2I+1.2S+1.2W &  231.0 &   15.5 &   1.2 \\
B &              114 (Final) Cb5.10-D+1.4W &  226.6 &    4.0 &  65.8 \\
C &                       60 Snow Drift 90 &  925.0 &   -0.8 &  -5.6 \\
C &               105 (Final) Cb5.1-D+1.4W &  358.3 &   -0.3 &  -7.2 \\
C &  80 (Final) Cb3.8-1.2D+1.2I+1.2RI+1.2W &  693.1 &   -2.2 &  -5.5 \\
C &  76 (Final) Cb3.4-1.2D+1.2I+1.2RI+1.2W &  691.5 &   -0.6 &  -8.4 \\
z &  73 (Final) Cb3.1-

C:\Users\rmacdonald.BLYTHANDBLYTH\AppData\Local\Temp\ipykernel_5520\99942857.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  summary_table_df = summary_table_df.append(support.df.iloc[0])
C:\Users\rmacdonald.BLYTHANDBLYTH\AppData\Local\Temp\ipykernel_5520\99942857.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  support.df.sort_values(by=i, ascending=False, inplace=True, key=abs)
C:\Users\rmacdonald.BLYTHANDBLYTH\AppData\Local\Temp\ipykernel_5520\99942857.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  summary_table_df = summary_table_df.append(support.df.iloc[0])
C:\Users\rmacdonald.BLYTHANDBLYTH\AppData\Local\Te